In [1]:
nn_model_id = 'retrained_sherlock7'

In [2]:
import pandas as pd
import numpy as np

from sherlock.deploy.train_sherlock import train_sherlock
from sherlock.deploy.predict_sherlock import predict_sherlock
from datetime import datetime
from sklearn.metrics import f1_score

### Load training and validation sets

In [3]:
start = datetime.now()
print(f'Started at {start}')

X_train = pd.read_parquet('train.parquet')
y_train = pd.read_parquet('../data/raw/train_labels.parquet').values.flatten()

print(f'Load data (train) process took {datetime.now() - start} seconds.')

Started at 2021-01-06 21:01:43.675077
Load data (train) process took 0:00:05.575578 seconds.


In [4]:
print('Distinct types for columns in the Dataframe (should be all float32):')
print(set(X_train.dtypes))

Distinct types for columns in the Dataframe (should be all float32):
{dtype('float32')}


In [5]:
start = datetime.now()
print(f'Started at {start}')

X_validation = pd.read_parquet('validation.parquet')
y_validation = pd.read_parquet('../data/raw/val_labels.parquet').values.flatten()

print(f'Load data (validation) process took {datetime.now() - start} seconds.')

Started at 2021-01-06 21:01:49.269459
Load data (validation) process took 0:00:01.843722 seconds.


### Re-train Sherlock

In [6]:
start = datetime.now()
print(f'Started at {start}')

train_sherlock(X_train, y_train, X_validation, y_validation, nn_id=nn_model_id);

print('Trained and saved new model.')
print(f'Finished at {datetime.now()}, took {datetime.now() - start} seconds')

Started at 2021-01-06 21:01:51.118531


W0106 21:01:51.328581 4362792384 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project/venv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0106 21:01:51.329710 4362792384 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project/venv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0106 21:01:51.333201 4362792384 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project/venv/lib/python3.7/site-pa

Successfully loaded and compiled model, now fitting model on data.
Train on 412059 samples, validate on 137353 samples
Epoch 1/100
412059/412059 [==============================] - 69s 167us/sample - loss: 1.6057 - categorical_accuracy: 0.6982 - val_loss: 1.0432 - val_categorical_accuracy: 0.8287
Epoch 2/100
412059/412059 [==============================] - 64s 154us/sample - loss: 0.9638 - categorical_accuracy: 0.8359 - val_loss: 0.9435 - val_categorical_accuracy: 0.8477
Epoch 3/100
412059/412059 [==============================] - 62s 151us/sample - loss: 0.8510 - categorical_accuracy: 0.8592 - val_loss: 0.8872 - val_categorical_accuracy: 0.8582
Epoch 4/100
412059/412059 [==============================] - 64s 154us/sample - loss: 0.7858 - categorical_accuracy: 0.8713 - val_loss: 0.8535 - val_categorical_accuracy: 0.8628
Epoch 5/100
412059/412059 [==============================] - 64s 155us/sample - loss: 0.7376 - categorical_accuracy: 0.8796 - val_loss: 0.8304 - val_categorical_accuracy

412059/412059 [==============================] - 63s 154us/sample - loss: 0.3266 - categorical_accuracy: 0.9369 - val_loss: 0.6139 - val_categorical_accuracy: 0.8940
Epoch 47/100
412059/412059 [==============================] - 63s 154us/sample - loss: 0.3241 - categorical_accuracy: 0.9372 - val_loss: 0.6088 - val_categorical_accuracy: 0.8938
Epoch 48/100
412059/412059 [==============================] - 63s 153us/sample - loss: 0.3219 - categorical_accuracy: 0.9378 - val_loss: 0.6190 - val_categorical_accuracy: 0.8940
Epoch 49/100
412059/412059 [==============================] - 64s 154us/sample - loss: 0.3213 - categorical_accuracy: 0.9376 - val_loss: 0.6025 - val_categorical_accuracy: 0.8942
Epoch 50/100
412059/412059 [==============================] - 64s 155us/sample - loss: 0.3185 - categorical_accuracy: 0.9383 - val_loss: 0.6116 - val_categorical_accuracy: 0.8940
Epoch 51/100
412059/412059 [==============================] - 64s 154us/sample - loss: 0.3171 - categorical_accuracy: 

### Load test set

In [7]:
start = datetime.now()
print(f'Started at {start}')

X_test = pd.read_parquet('test.parquet')
y_test = pd.read_parquet('../data/raw/test_labels.parquet').values.flatten()

print('Trained and saved new model.')
print(f'Finished at {datetime.now()}, took {datetime.now() - start} seconds')

Started at 2021-01-06 21:59:21.879825
Trained and saved new model.
Finished at 2021-01-06 21:59:23.780420, took 0:00:01.900605 seconds


### Make prediction

In [8]:
predicted_labels = predict_sherlock(X_test, nn_id=nn_model_id)

In [9]:
print(f'prediction count {len(predicted_labels)}, type = {type(predicted_labels)}')

size=len(y_test)

# Should be fully deterministic too.
f1_score(y_test[:size], predicted_labels[:size], average="weighted")

prediction count 137353, type = <class 'numpy.ndarray'>


0.8948361495992552

In [10]:
# Best results
# 0.8948361495992552

# LAST
# 0.8941992181577104
# 0.8661276466591463
# 0.8765360861865846
# 0.8777825108666634
# 0.8709458033528152

## Review errors

In [11]:
from collections import Counter

size = len(y_test)
mismatches = list()

for idx, k1 in enumerate(y_test[:size]):
    k2 = predicted_labels[idx]

    if k1 != k2:
        mismatches.append(k1)
#        if k1 in ('brand'):
#        print(f'[{idx}] expected "{k1}" but predicted "{k2}"')
        
f1 = f1_score(y_test[:size], predicted_labels[:size], average="weighted")
print(f'Total mismatches: {len(mismatches)} (F1 score: {f1})')

data = Counter(mismatches)
data.most_common()   # Returns all unique items and their counts

Total mismatches: 14466 (F1 score: 0.8948361495992552)


[('rank', 702),
 ('name', 682),
 ('position', 525),
 ('location', 495),
 ('region', 492),
 ('description', 453),
 ('type', 387),
 ('album', 373),
 ('artist', 370),
 ('team', 366),
 ('area', 355),
 ('product', 342),
 ('category', 335),
 ('notes', 307),
 ('ranking', 306),
 ('team Name', 295),
 ('city', 292),
 ('code', 291),
 ('company', 287),
 ('gender', 262),
 ('class', 261),
 ('day', 250),
 ('person', 230),
 ('owner', 226),
 ('status', 207),
 ('weight', 201),
 ('brand', 185),
 ('sex', 183),
 ('year', 183),
 ('result', 182),
 ('service', 181),
 ('credit', 180),
 ('manufacturer', 177),
 ('age', 167),
 ('country', 166),
 ('order', 163),
 ('address', 158),
 ('duration', 155),
 ('plays', 152),
 ('range', 150),
 ('component', 150),
 ('sales', 147),
 ('origin', 147),
 ('county', 134),
 ('state', 133),
 ('format', 127),
 ('club', 119),
 ('nationality', 106),
 ('director', 104),
 ('capacity', 101),
 ('command', 97),
 ('language', 90),
 ('affiliation', 89),
 ('publisher', 87),
 ('symbol', 86),
 

In [12]:
test_samples = pd.read_parquet('../data/raw/test_values.parquet')

In [13]:
from ast import literal_eval

idx = 541
original = test_samples.iloc[idx]
converted = original.apply(literal_eval).to_list()

print(f'Predicted "{predicted_labels[idx]}", actual label "{y_test[idx]}". Actual values:\n{converted}')

Predicted "age", actual label "position". Actual values:
[[2, 4]]
